In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np

# Define LeNet-5 model without dropout for color images
class LeNet5Color(nn.Module):
    def __init__(self):
        super(LeNet5Color, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 5 * 5, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.fc1(x)
        return x

# Custom hook to register activations for conv1 layer only
class Conv1ActivationHook:
    def __init__(self):
        self.activations = []

    def __call__(self, module, input, output):
        self.activations.append(output.cpu().detach().numpy())

# Load CIFAR-10 dataset for color images
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# Loss function
criterion_color = nn.CrossEntropyLoss()


# Train the model for color images
for training_iteration in range(10):  # Train the model 10 times

    net_color = LeNet5Color()

    optimizer_color = optim.Adam(net_color.parameters(), lr=0.001, weight_decay=0.001)

    # Register hook for activation patterns for conv1 layer only
    activation_hook = Conv1ActivationHook()
    hook_handle = net_color.conv1.register_forward_hook(activation_hook)

    # Lists to store training and testing accuracies for each epoch
    training_accuracies = []
    testing_accuracies = []

    for epoch in range(10):  # Change the number of epochs as needed
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer_color.zero_grad()
            outputs = net_color(inputs)
            loss = criterion_color(outputs, labels)
            loss.backward()
            optimizer_color.step()
            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_accuracy = correct_train / total_train
        training_accuracies.append(train_accuracy)

        train_loss = running_loss / len(trainloader)
        print(f'Training Iteration {training_iteration + 1}, Epoch {epoch + 1}/{10}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}')

        # Test the model for color images
        correct_test = 0
        total_test = 0
        test_loss = 0.0

        with torch.no_grad():
            for data in testloader:
                inputs, labels = data
                outputs = net_color(inputs)
                loss = criterion_color(outputs, labels)
                test_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

        test_accuracy = correct_test / total_test
        testing_accuracies.append(test_accuracy)

        test_loss /= len(testloader)
        print(f'Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}')

    # Save activation patterns for conv1 layer for each run
   # activation_filename = f'conv1_activations3_{training_iteration}_{epoch}.npy'
    #np.save(activation_filename, np.array(activation_hook.activations))

    # Save the model for each run
    model_filename = f'model3_{training_iteration}_{epoch}.pth'
    torch.save(net_color.state_dict(), model_filename)

    # Save training and testing activities for each run
    training_filename = f'training_activities3_{training_iteration}_{epoch}.npy'
    testing_filename = f'testing_activities3_{training_iteration}_{epoch}.npy'

    np.save(training_filename, np.array(training_accuracies))
    np.save(testing_filename, np.array(testing_accuracies))

    # Remove the hook after each epoch
    hook_handle.remove()


Files already downloaded and verified
Files already downloaded and verified
Training Iteration 1, Epoch 1/10, Training Loss: 1.5098, Training Accuracy: 0.4555
Testing Loss: 1.3359, Testing Accuracy: 0.5213
Training Iteration 1, Epoch 2/10, Training Loss: 1.3015, Training Accuracy: 0.5400
Testing Loss: 1.2368, Testing Accuracy: 0.5625
Training Iteration 1, Epoch 3/10, Training Loss: 1.2359, Training Accuracy: 0.5641
Testing Loss: 1.1999, Testing Accuracy: 0.5779
Training Iteration 1, Epoch 4/10, Training Loss: 1.2029, Training Accuracy: 0.5771
Testing Loss: 1.1944, Testing Accuracy: 0.5843
Training Iteration 1, Epoch 5/10, Training Loss: 1.1814, Training Accuracy: 0.5884
Testing Loss: 1.2664, Testing Accuracy: 0.5606
Training Iteration 1, Epoch 6/10, Training Loss: 1.1647, Training Accuracy: 0.5916
Testing Loss: 1.1800, Testing Accuracy: 0.5867
Training Iteration 1, Epoch 7/10, Training Loss: 1.1504, Training Accuracy: 0.5990
Testing Loss: 1.2360, Testing Accuracy: 0.5646
Training Itera